In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

from torchvision.models import resnet18, ResNet18_Weights
import torch
import PIL

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import numpy as np
from PIL import Image

# DATASET_PATH = '/kaggle/input/dog-emotion/Dog Emotion/'
DATASET_PATH = '/content/drive/MyDrive/Dog_Emotion_1/'

data = {'img': [], 'emotion': []}

for emotion in ['angry', 'happy', 'relaxed', 'sad']:
    for dirname, _, filenames in os.walk(DATASET_PATH + emotion + '/'):
        for filename in filenames:
            data['img'].append(os.path.join(dirname, filename))
            data['emotion'].append(emotion)

df = pd.DataFrame.from_dict(data)

In [4]:
df

,img,emotion
0,/content/drive/MyDrive/Dog_Emotion_1/angry/0Av...,angry
1,/content/drive/MyDrive/Dog_Emotion_1/angry/0C5...,angry
2,/content/drive/MyDrive/Dog_Emotion_1/angry/09d...,angry
3,/content/drive/MyDrive/Dog_Emotion_1/angry/0TS...,angry
4,/content/drive/MyDrive/Dog_Emotion_1/angry/0aN...,angry
...,...,...
4005,/content/drive/MyDrive/Dog_Emotion_1/sad/zvgEE...,sad
4006,/content/drive/MyDrive/Dog_Emotion_1/sad/ztwzd...,sad
4007,/content/drive/MyDrive/Dog_Emotion_1/sad/zmAi6...,sad
4008,/content/drive/MyDrive/Dog_Emotion_1/sad/zq3lf...,sad


In [5]:
emotion_labels_map = {}

unique_emotions = df['emotion'].unique()
for i in range(len(unique_emotions)):
    emotion_labels_map[unique_emotions[i]] = i

In [6]:
df['emotion_label'] = df['emotion'].apply(lambda x: emotion_labels_map[x])
df

,img,emotion,emotion_label
0,/content/drive/MyDrive/Dog_Emotion_1/angry/0Av...,angry,0
1,/content/drive/MyDrive/Dog_Emotion_1/angry/0C5...,angry,0
2,/content/drive/MyDrive/Dog_Emotion_1/angry/09d...,angry,0
3,/content/drive/MyDrive/Dog_Emotion_1/angry/0TS...,angry,0
4,/content/drive/MyDrive/Dog_Emotion_1/angry/0aN...,angry,0
...,...,...,...
4005,/content/drive/MyDrive/Dog_Emotion_1/sad/zvgEE...,sad,3
4006,/content/drive/MyDrive/Dog_Emotion_1/sad/ztwzd...,sad,3
4007,/content/drive/MyDrive/Dog_Emotion_1/sad/zmAi6...,sad,3
4008,/content/drive/MyDrive/Dog_Emotion_1/sad/zq3lf...,sad,3


In [7]:
from torch.utils.data import DataLoader, Dataset

class Dataset(Dataset):
    def __init__(self, data, transform):
        self.data = data
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image = PIL.Image.open(self.data.loc[idx, "img"]).convert('RGB')
        image = self.transform(image)
        label = torch.tensor(self.data.loc[idx, "emotion_label"])
        return image, label

In [8]:
from torchvision.transforms import v2
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df["img"], df["emotion_label"], random_state=42)

X_train.index = np.arange(len(X_train))
y_train.index = np.arange(len(y_train))
X_test.index = np.arange(len(X_test))
y_test.index = np.arange(len(y_test))

In [36]:
transform_train = transform_test = v2.Compose([
    v2.Resize(size=(224, 224)),
    v2.PILToTensor(),
    v2.ToDtype(torch.float32, scale=True),
    v2.RandomResizedCrop(224),  # Random crop with resize to 224x224
    v2.RandomHorizontalFlip(p=0.5),  # Horizontal flip with 50% probability
    v2.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Random color adjustments
    v2.RandomRotation(7),
    v2.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

transform_test = v2.Compose([
    v2.Resize(size=(224, 224)),
    v2.PILToTensor(),
    v2.ToDtype(torch.float32, scale=True),
    v2.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [37]:
train_ds = Dataset(pd.concat([X_train, y_train], axis=1), transform_train)
test_ds = Dataset(pd.concat([X_test, y_test], axis=1), transform_test)


In [42]:
from torchvision import models
import torch.nn as nn

efficientnet = models.efficientnet_b0(weights=models.EfficientNet_B0_Weights.DEFAULT)
efficientnet.classifier[1] = nn.Linear(efficientnet.classifier[1].in_features, 4)

resnet50 = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
resnet50.fc = nn.Linear(resnet50.fc.in_features, 4)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
efficientnet = efficientnet.to(device)
resnet50 = resnet50.to(device)

Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth
100%|██████████| 20.5M/20.5M [00:00<00:00, 91.4MB/s]
Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 169MB/s]


In [43]:
import torch
import torch.nn as nn

class MultiModelExtraction(nn.Module):
    def __init__(self, efficientnet, resnet50):
        super(MultiModelExtraction, self).__init__()
        self.efficientnet = efficientnet
        self.resnet50 = resnet50

        self.efficientnet.classifier = nn.Identity()
        self.resnet50.fc = nn.Identity()

        self.classifier = nn.Linear(1280 + 2048, 4)

    def forward(self, x):
        features_eff = self.efficientnet(x)
        features_res = self.resnet50(x)

        combined_features = torch.cat((features_eff, features_res), dim=1)

        output = self.classifier(combined_features)
        return output

multi_model = MultiModelExtraction(efficientnet, resnet50).to(device)

In [44]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(multi_model.parameters(), lr=3e-4)

num_epochs = 10
batch_size = 32

train_loader = DataLoader(train_ds, batch_size=batch_size, drop_last=True, shuffle=True)
test_loader = DataLoader(test_ds, batch_size=batch_size)

for epoch in range(num_epochs):
    multi_model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        outputs = multi_model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        if (i+1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], '
                  f'Loss: {running_loss/100:.4f}, '
                  f'Accuracy: {100 * correct/total:.2f}%')
            running_loss = 0.0

print('Training complete')

Epoch [1/10], Step [10/93], Loss: 0.1316, Accuracy: 40.94%
Epoch [1/10], Step [20/93], Loss: 0.1191, Accuracy: 43.75%
Epoch [1/10], Step [30/93], Loss: 0.1011, Accuracy: 48.23%
Epoch [1/10], Step [40/93], Loss: 0.0993, Accuracy: 50.39%
Epoch [1/10], Step [50/93], Loss: 0.0851, Accuracy: 53.25%
Epoch [1/10], Step [60/93], Loss: 0.0721, Accuracy: 56.09%
Epoch [1/10], Step [70/93], Loss: 0.0837, Accuracy: 57.77%
Epoch [1/10], Step [80/93], Loss: 0.0695, Accuracy: 59.38%
Epoch [1/10], Step [90/93], Loss: 0.0714, Accuracy: 60.62%
Epoch [2/10], Step [10/93], Loss: 0.0770, Accuracy: 70.31%
Epoch [2/10], Step [20/93], Loss: 0.0680, Accuracy: 71.09%
Epoch [2/10], Step [30/93], Loss: 0.0658, Accuracy: 72.40%
Epoch [2/10], Step [40/93], Loss: 0.0679, Accuracy: 72.66%
Epoch [2/10], Step [50/93], Loss: 0.0673, Accuracy: 72.31%
Epoch [2/10], Step [60/93], Loss: 0.0610, Accuracy: 72.76%
Epoch [2/10], Step [70/93], Loss: 0.0715, Accuracy: 72.14%
Epoch [2/10], Step [80/93], Loss: 0.0608, Accuracy: 72.5

In [45]:
multi_model.eval()

predictions = []
true_labels = []

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = multi_model(images)
        _, predicted = torch.max(outputs.data, 1)
        predictions += predicted.cpu().tolist()
        true_labels += labels.cpu().tolist()

from sklearn.metrics import accuracy_score
accuracy = accuracy_score(predictions, true_labels)
print(f'Test Accuracy: {accuracy * 100:.2f}%')

Test Accuracy: 84.25%
